In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
train_dir = './dataset/train'
validation_dir = './dataset/validation'

im_size = 224
INIT_LR = 1e-5

vgg_conv = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(im_size, im_size, 3)
)

In [3]:
vgg_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [4]:
for layer in vgg_conv.layers[:-4]: # last 4 layers will be trainable
    layer.trainable = False

for layer in vgg_conv.layers:
    print(layer, layer.trainable)

print('total layers:', vgg_conv.layers.__len__())

<keras.engine.input_layer.InputLayer object at 0x7f35b43ee4e0> False
<keras.layers.convolutional.Conv2D object at 0x7f35b43ee860> False
<keras.layers.convolutional.Conv2D object at 0x7f35b43ee828> False
<keras.layers.pooling.MaxPooling2D object at 0x7f35b440b908> False
<keras.layers.convolutional.Conv2D object at 0x7f35b4424828> False
<keras.layers.convolutional.Conv2D object at 0x7f35b43be978> False
<keras.layers.pooling.MaxPooling2D object at 0x7f35b43e9128> False
<keras.layers.convolutional.Conv2D object at 0x7f35b43e9780> False
<keras.layers.convolutional.Conv2D object at 0x7f35b4395080> False
<keras.layers.convolutional.Conv2D object at 0x7f35b432d588> False
<keras.layers.pooling.MaxPooling2D object at 0x7f35b42ed320> False
<keras.layers.convolutional.Conv2D object at 0x7f35b435b438> False
<keras.layers.convolutional.Conv2D object at 0x7f35b4316da0> False
<keras.layers.convolutional.Conv2D object at 0x7f35b42c52e8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f35b42e0940>

In [5]:
model = Sequential()

for layer in vgg_conv.layers: # without the last dense layer (that is the predict layer)
	model.add(layer)

model.add(Flatten())
model.add(Dense(2**13))
model.add(Dense(2**9))
model.add(Dense(2, activation='softmax', name='predictions'))

model.summary()

del vgg_conv

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [6]:
# Data augmentation

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Change the batchsize according to your system RAM
train_batchsize = 20
val_batchsize = 20

# Data Generator for Training data
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(im_size, im_size),
        batch_size=train_batchsize,
        class_mode='categorical'
)

# Data Generator for Validation data
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(im_size, im_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False
)

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [7]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(lr=INIT_LR),
    metrics=['acc']
)

# Train the Model
# NOTE that we have multiplied the steps_per_epoch by 2. This is because we are using data augmentation.
history = model.fit_generator(
      train_generator,
      steps_per_epoch=2*train_generator.samples/train_generator.batch_size ,
      epochs=40,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1
)

# Save the Model
model.save('da_last4_layers.h5')

# Plot the accuracy and loss curves
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Epoch 1/40


ResourceExhaustedError: OOM when allocating tensor with shape[25088,8192] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/SGD/mul_12 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](SGD/momentum/read, training/SGD/Variable_6/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training/SGD/mul_12', defined at:
  File "/home/choppy/miniconda3/envs/choppy/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/choppy/miniconda3/envs/choppy/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/choppy/miniconda3/envs/choppy/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/choppy/miniconda3/envs/choppy/lib/python3.6/site-packages/traitlets/config